# Script: Replace all 0's with Nodata (nan's)

### Florian Beyer

2019-03-13

Version 0.1


In [6]:
# required packages
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

import glob
import os
import sys

import numpy as np

In [7]:
# folder with all images
infolder = r'N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\006_Subsets_wolkenfrei_V2'

# folder where to save the corected images
outfolder = r'N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\007_0_to_NaN\\'

In [8]:
def convert_0_to_NaN(image):
    '''
    This function takes a rasterio multiband image and replacec all 0 with NoData (np.nan)
    Output is a List of np.arrays (bands)
    
    image: rasterio dataset
    '''
    bands_container = []
    for i in range(image.count): # for loop to read all bands
        image_band = image.read(i+1) # read band
        image_band = image_band.astype(np.float32) # nan's have to be float
        image_band[image_band == 0] = np.nan # replacing 0s with nan's
        bands_container.append(image_band)
    return bands_container # final list of all bands

In [15]:
def save_list_to_geotiff(bandslist, source_image, save_path, out_filename):
    '''
    This function saves all the corrected images
    
    bandlist: list of numpy arrays for every Band
    source_image: rasterio dataset
    savepath: string with the path where the image will be saved
    outfile_name: filename of the corrected image
    '''
    output_path=save_path+out_filename # new output path and image name
    with rasterio.open(output_path,
                       'w',
                       driver='GTiff',
                       height=bandslist[0].shape[0],
                       width=bandslist[0].shape[1],
                       count=source_image.count,
                       crs=source_image.crs,
                       transform=source_image.transform,
                       dtype=rasterio.float32
                      ) as dst:
        for i in range(len(bandslist)): # loop over all bands during writing
            dst.write(bandslist[i],i+1) # write on disk

In [10]:
# search all datasets which end with *.tif
temp = os.path.join(infolder, '*.tif')
files = glob.glob(temp) # input entire path in front of the image
temp = None
files

['N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\006_Subsets_wolkenfrei_V2\\20180420_S2B_MSIL2A_T32UPE_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\006_Subsets_wolkenfrei_V2\\20180420_S2B_MSIL2A_T32UPF_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\006_Subsets_wolkenfrei_V2\\20180420_S2B_MSIL2A_T33UUA_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\006_Subsets_wolkenfrei_V2\\20180420_S2B_MSIL2A_T33UUV_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\006_Subsets_wolkenfrei_V2\\20180505_S2A_MSIL2A_T32UPF_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\006_Subsets_wolkenfrei_V2\\20180505_S2A_MSIL2A_T33UUA_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\006_Subsets_wolkenfrei_V2\\20180505_S2A_MSIL2A_T33UUV_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\006_Subsets_wolkenfrei_V2\\20180507_S2B_MSIL2A_T32UPE_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\006_Subsets_wolken

In [11]:
# read all dataset with rasterio in a list
files_to_convert = []
for i in files:
    src = rasterio.open(i)
    files_to_convert.append(src)
files_to_convert

[<open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\006_Subsets_wolkenfrei_V2\20180420_S2B_MSIL2A_T32UPE_s.tif' mode='r'>,
 <open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\006_Subsets_wolkenfrei_V2\20180420_S2B_MSIL2A_T32UPF_s.tif' mode='r'>,
 <open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\006_Subsets_wolkenfrei_V2\20180420_S2B_MSIL2A_T33UUA_s.tif' mode='r'>,
 <open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\006_Subsets_wolkenfrei_V2\20180420_S2B_MSIL2A_T33UUV_s.tif' mode='r'>,
 <open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\006_Subsets_wolkenfrei_V2\20180505_S2A_MSIL2A_T32UPF_s.tif' mode='r'>,
 <open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\006_Subsets_wolkenfrei_V2\20180505_S2A_MSIL2A_T33UUA_s.tif' mode='r'>,
 <open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\006_Subsets_wolkenfrei_V2\20180505_S2A_MSIL2A_T33UUV_s.tif' mode='r'>,
 <open

In [17]:
for i in range(len(files_to_convert)):
    '''
    loop over the opened raterio datasets in the list and 
    1. replace all 0's with nan's
    2. save the new images to the output directory with new names
    '''
    temp = convert_0_to_NaN(image=files_to_convert[i])
    
    filename = files[i][-32:-4]+'_to_NaN.tif'
    
    save_list_to_geotiff(bandslist=temp,
                         source_image=files_to_convert[i],
                         save_path=outfolder,
                         out_filename=filename)
    print '{} von {} images saved: {}'.format(i+1,len(files_to_convert)+1,filename)

1 von 80 images saved: 20180420_S2B_MSIL2A_T32UPE_s_to_NaN.tif
2 von 80 images saved: 20180420_S2B_MSIL2A_T32UPF_s_to_NaN.tif
3 von 80 images saved: 20180420_S2B_MSIL2A_T33UUA_s_to_NaN.tif
4 von 80 images saved: 20180420_S2B_MSIL2A_T33UUV_s_to_NaN.tif
5 von 80 images saved: 20180505_S2A_MSIL2A_T32UPF_s_to_NaN.tif
6 von 80 images saved: 20180505_S2A_MSIL2A_T33UUA_s_to_NaN.tif
7 von 80 images saved: 20180505_S2A_MSIL2A_T33UUV_s_to_NaN.tif
8 von 80 images saved: 20180507_S2B_MSIL2A_T32UPE_s_to_NaN.tif
9 von 80 images saved: 20180507_S2B_MSIL2A_T32UPF_s_to_NaN.tif
10 von 80 images saved: 20180507_S2B_MSIL2A_T33UUA_s_to_NaN.tif
11 von 80 images saved: 20180507_S2B_MSIL2A_T33UUV_s_to_NaN.tif
12 von 80 images saved: 20180520_S2B_MSIL2A_T32UPE_s_to_NaN.tif
13 von 80 images saved: 20180520_S2B_MSIL2A_T32UPF_s_to_NaN.tif
14 von 80 images saved: 20180520_S2B_MSIL2A_T33UUA_s_to_NaN.tif
15 von 80 images saved: 20180520_S2B_MSIL2A_T33UUV_s_to_NaN.tif
16 von 80 images saved: 20180530_S2B_MSIL2A_T32UP

In [18]:
# for one single image:
src = rasterio.open(r'N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\005_Convert_UTM32_to_UTM33\\20180505_S2A_MSIL2A_T32UPE_s_reproject.tif')

In [19]:
# for one single image:
temp = convert_0_to_NaN(image=src)
    
filename = '20180505_S2A_MSIL2A_T32UPE_s_to_NaN.tif'

save_list_to_geotiff(bandslist=temp,
                     source_image=src,
                     save_path='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\007_0_to_NaN\\',
                     out_filename=filename)